In [1]:
#!pip install selenium

In [2]:
#!pip install webdriver-manager

In [3]:
import selenium
from selenium import webdriver
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time 
import re

In [4]:
# Create driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Get the page url 
page_url = "https://gilmoregirls.fandom.com/wiki/Category:Characters"
driver.get(page_url) # Open the page 
wait = WebDriverWait(driver, 10) # Wait until the page renders

[WDM] - Downloading: 100%|█████████████████| 8.41M/8.41M [00:00<00:00, 11.2MB/s]
/var/folders/_b/l_819g5924d_y00npn9gt60h0000gn/T/ipykernel_29962/343686796.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
# Accept the cookies 
driver.find_element(By.XPATH, "//div[text()='AKCEPTUJĘ']").click()

In [6]:
# Create a list of characters
character_element = driver.find_elements(By.CLASS_NAME, 'category-page__member-link')

In [2]:
# Append characters to the list 
characters = []
for value in character_element[8:]:
    characters_list = value.text
    characters.append({'full name' : characters_list})

NameError: name 'character_element' is not defined

In [3]:
# Create a dataframe 
characters_df = pd.DataFrame(characters)

# Separate name from the full name 
characters['name'] = characters['full name'].apply(lambda x: x if 
                                                   x.startswith(("Ms.", "Miss", "Mrs.", "Mr")) else x.split()[0]) 

# Correct spelling mistakes 
characters_df.at[24, 'name'] = 'Chris'
characters_df.at[62, 'name'] = 'Digger'
characters_df.at[84, 'name'] = 'Trix'
characters_df.at[152, 'name'] = 'Chui'



NameError: name 'pd' is not defined

In [15]:
# save to csv
characters_df.to_csv('characters_list.csv', sep='\t', index=False)

NameError: name 'characters_df' is not defined